In [1]:
# load in packages required for analysis.

%matplotlib qt
import os.path

from refnx.dataset import ReflectDataset
from refnx.analysis import Transform, CurveFitter, Objective, GlobalObjective, Parameter
from refnx.reflect import SLD, ReflectModel, MixedReflectModel
from MixedLayerSlab import MixedMagSlabs2 #load in custom model for magnetic layers in non-polarised instrument.
import refnx

import dynesty

import scipy
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#print what versions we are using.

print('refnx: %s\nscipy: %s\nnumpy: %s' % (refnx.version.version, scipy.version.version, np.version.version))

refnx: 0.1.17
scipy: 1.5.2
numpy: 1.19.2


In [3]:
#load data from .txt files - this can be done smarter but works as a test.

file_path_hdod = '61112_14.txt'
file_path_ddod = '61143_45.txt'

#Define datasets with data loaded

P1hdod = ReflectDataset(file_path_hdod)
P1ddod = ReflectDataset(file_path_ddod)

In [4]:
#now define some SLDs we will use...

#LAYERS#
Si = SLD(2.07, name='Si')
SiO2 = SLD(3.47, name='SiO2')
Fe = SLD(8.02, name='Fe')
FeOx = SLD(7.0, name='FeOx')
Adv_J = SLD(0, name='Adv_J')

#SOLVENTS#
hdod_SLD = SLD(-0.462, name='hdod1')
ddod_SLD = SLD(6.7, name='ddod1')

In [5]:
"""
Here we define the material layers at the interface.
"""

#Create slabs for model

##SiO2##

SiO2_lay = SiO2(15, 3)

SiO2_lay.thick.setp(vary=True, bounds=(1, 25))

##Fe##
# share the same Fe thickness, SiO2-Fe roughness - we specify variation of parameters here.
SiO2_Fe_R = Parameter(4, 'SiO2_Fe_R', vary=True, bounds=(1, 10))
Fe.real.setp(vary=True, bounds=(7.5, 8.1))
Fe_thick = Parameter(190, 'Fe_t', vary=True, bounds=(170, 210))
Fe_magmom = Parameter(2.1, 'Fe_magmom', vary=True, bounds=(1.9, 2.2))
Fe_ScattLen = Parameter(0.0000945, 'Fe_SL', vary=False)

FeUp_lay = MixedMagSlabs2.USM_Slab(Fe_thick, Fe, Fe_magmom, Fe_ScattLen, SiO2_Fe_R)
FeDown_lay = MixedMagSlabs2.DSM_Slab(Fe_thick, Fe, Fe_magmom, Fe_ScattLen, SiO2_Fe_R)

##FeOx##
Fe_FeOx_R = Parameter(5, 'FeOx_Fe_R', vary=True, bounds=(1, 15))
FeOx.real.setp(vary=True, bounds=(5.0, 7.2))
FeOx_thick = Parameter(30, 'FeOx_t', vary=True, bounds=(20, 40))
FeOx_m = Parameter(0.5, 'FeOx_m', vary=True, bounds=(0, 1.3))

FeOxUp_lay = MixedMagSlabs2.USM_nomagmom_Slab(FeOx_thick, FeOx, FeOx_m, Fe_FeOx_R)
FeOxDown_lay = MixedMagSlabs2.DSM_nomagmom_Slab(FeOx_thick, FeOx, FeOx_m, Fe_FeOx_R)

# INTERFACIAL LAYERS#

# ADV #
# share the same FeOx_R across all interfacial layers.
FeOx_R = Parameter(6, 'FeOx_R', vary=True, bounds=(1, 15))

Adv_thick = Parameter(10, 'Adv_t', vary=True, bounds=(1, 20))

#This adv_layer model uses one SLD for hdod and ddod contrast with one solvation parameter.
Adv_J_lay = Adv_J(Adv_thick, FeOx_R)
Adv_J_lay.vfsolv.setp(vary=True, bounds=(0, 1))

Adv_J_lay.sld.real.setp(vary=True, bounds=(-0.54, 6))

###INTF-SOLV ROUGHNESS###

# share the Adv-solv roughness.
Adv_dod_R = Parameter(7, 'Adv_dod_R', vary=True, bounds=(1, 15))

### SOLVENT PARAMS FOR ADV_LAYER SOLVENT

hdod_adv_lay = hdod_SLD(0, Adv_dod_R)
ddod_adv_lay = ddod_SLD(0, Adv_dod_R)

ddod_adv_lay.sld.real.setp(vary=True, bounds=(5, 6.703))
hdod_adv_lay.sld.real.setp(vary=False)

In [6]:
P1_hdod_U = Si | SiO2_lay | FeUp_lay | FeOxUp_lay | Adv_J_lay | hdod_adv_lay
P1_hdod_D = Si | SiO2_lay | FeDown_lay | FeOxDown_lay | Adv_J_lay | hdod_adv_lay

P1_ddod_U = Si | SiO2_lay | FeUp_lay | FeOxUp_lay | Adv_J_lay | ddod_adv_lay
P1_ddod_D = Si | SiO2_lay | FeDown_lay | FeOxDown_lay | Adv_J_lay | ddod_adv_lay

In [7]:
#now define the models

#these are the scale and background parameters
P1hdod_M_int = Parameter(0.5, 'P1hdod_M_int', vary=True, bounds=(0.5*0.8, 0.5*1.2))
P1ddod_M_int = Parameter(0.5, 'P1ddod_M_int', vary=True, bounds=(0.5*0.8, 0.5*1.2))
P1hdod_M_bkg = Parameter(1e-6, 'P1hdod_M_bkg', vary=True, bounds=(1e-7, 2e-5))
P1ddod_M_bkg = Parameter(1e-6, 'P1ddod_M_bkg', vary=True, bounds=(1e-7, 2e-5))

#define adv_layer interface objective & global objective. These adv layers have their own SLD.
P1hdod_M_J_adv = MixedReflectModel((P1_hdod_U, P1_hdod_D), scales=(P1hdod_M_int, P1hdod_M_int), bkg=P1hdod_M_bkg, dq=4.7096)
P1ddod_M_J_adv = MixedReflectModel((P1_ddod_U, P1_ddod_D), scales=(P1ddod_M_int, P1ddod_M_int), bkg=P1ddod_M_bkg, dq=4.7096)
P1hdod_J_adv_obj = Objective(P1hdod_M_J_adv, P1hdod, transform=Transform('logY'))
P1ddod_J_adv_obj = Objective(P1ddod_M_J_adv, P1ddod, transform=Transform('logY'))
J_adv_glob_obj = GlobalObjective([P1hdod_J_adv_obj, P1ddod_J_adv_obj])

In [8]:
### BE SENSIBLE AND HAVE A LOOK AT YOUR DATA BEFORE NS ###

P1hdod_J_adv_obj.plot()
P1ddod_J_adv_obj.plot()

### ALSO HAVE A LOOK AT THE SLD TO CHECK ALL LOOKS FINE ###

# plt.plot(*P1_ddod_U.sld_profile())
# plt.plot(*P1_ddod_D.sld_profile())
# plt.ylabel('SLD /$10^{-6} \AA^{-2}$')
# plt.xlabel('distance / $\AA$');

### makes sense to look at the list of varying parameters to ensure no mistakes ###
print(J_adv_glob_obj.varying_parameters().names())
print(J_adv_glob_obj)

['P1hdod_M_int', 'P1hdod_M_bkg', 'SiO2 - thick', 'Fe_t', 'Fe - sld', 'Fe_magmom', 'SiO2_Fe_R', 'FeOx_t', 'FeOx - sld', 'FeOx_m', 'FeOx_Fe_R', 'Adv_t', 'Adv_J - sld', 'FeOx_R', 'Adv_J - volfrac solvent', 'Adv_dod_R', 'P1ddod_M_int', 'P1ddod_M_bkg', 'ddod1 - sld']
_______________________________________________________________________________

--Global Objective--
________________________________________________________________________________
Objective - 2302950664648
Dataset = 61112_14
datapoints = 186
chi2 = 6664.41460778499
Weighted = True
Transform = Transform('logY')
________________________________________________________________________________
Parameters:       ''       
[Parameters(data=[Parameters(data=[Parameter(value=0.5, name='P1hdod_M_int', vary=True, bounds=Interval(lb=0.4, ub=0.6), constraint=None), Parameter(value=0.5, name='P1hdod_M_int', vary=True, bounds=Interval(lb=0.4, ub=0.6), constraint=None)], name='scale factors'), Parameter(value=1e-06, name='P1hdod_M_bkg', va

In [37]:
### initial fit to check the model works how we wish. ###
fitter = CurveFitter(J_adv_glob_obj)
fitter.fit('differential_evolution'); #initial fit. lets check the model is how we want. note this fit does not appear to be
print(J_adv_glob_obj)

187it [06:22,  2.05s/it]


_______________________________________________________________________________

--Global Objective--
________________________________________________________________________________
Objective - 1351881691784
Dataset = 61112_14
datapoints = 186
chi2 = 178.2488689410569
Weighted = True
Transform = Transform('logY')
________________________________________________________________________________
Parameters:       ''       
[Parameters(data=[Parameters(data=[Parameter(value=0.46979263277002653, name='P1hdod_M_int', vary=True, bounds=Interval(lb=0.4, ub=0.6), constraint=None), Parameter(value=0.46979263277002653, name='P1hdod_M_int', vary=True, bounds=Interval(lb=0.4, ub=0.6), constraint=None)], name='scale factors'), Parameter(value=8.003286216594622e-06, name='P1hdod_M_bkg', vary=True, bounds=Interval(lb=1e-07, ub=2e-05), constraint=None), Parameter(value=4.7096, name='dq - resolution', vary=False, bounds=Interval(lb=-np.inf, ub=np.inf), constraint=None)], name='instrument parameters')]


In [39]:
#check acceptable fits and SLD profiles...
P1hdod_J_adv_obj.plot()
P1ddod_J_adv_obj.plot()

# plt.plot(*P1_ddod_U.sld_profile())
# plt.plot(*P1_ddod_D.sld_profile())
# plt.ylabel('SLD /$10^{-6} \AA^{-2}$')
# plt.xlabel('distance / $\AA$');

(<Figure size 640x480 with 1 Axes>, <AxesSubplot:>)

In [40]:
### Nested Sampling ### Now we do the nested sampling!
nested_sampler = dynesty.NestedSampler(J_adv_glob_obj.logl, J_adv_glob_obj.prior_transform, ndim=len(J_adv_glob_obj.varying_parameters()))
nested_sampler.run_nested()

1747it [02:11,  3.08it/s, bound: 0 | nc: 88 | ncall: 17544 | eff(%):  9.958 | loglstar:   -inf < -4865.620 <    inf | logz: -4875.951 +/-  0.201 | dlogz: 4486.470 >  0.509] C:\Anaconda\envs\refnx\lib\site-packages\dynesty\sampling.py:216: UserWarning: Random number generation appears to be extremely inefficient. Adjusting the scale-factor accordingly.
  warnings.warn("Random number generation appears to be "
26258it [1:24:39,  5.17it/s, +500 | bound: 381 | nc: 1 | ncall: 630322 | eff(%):  4.245 | loglstar:   -inf < 856.786 <    inf | logz: 804.840 +/-  0.461 | dlogz:  0.001 >  0.509]


In [41]:
ns_results_sep_adv = nested_sampler.results
#print(ns_results_sep_adv)

from dynesty import plotting as dyplot

# Plot a summary of the run.
#rfig, raxes = dyplot.runplot(ns_results_sep_adv, logplot=True) #this doesn't seem to like evidences that are large.
#seems the plotting function plots evidence in a linear fashion, and hence tries to calculate e^(ln(evidence)).
#if ln(evidence) is large, i.e 800, then we get an overflow issue as this number is HUGE.

# Plot traces and 1-D marginalized posteriors.
#tfig, taxes = dyplot.traceplot(ns_results_sep_adv)

# Plot the 2-D marginalized posteriors.
#cfig, caxes = dyplot.cornerplot(ns_results_sep_adv, labels=sep_adv_glob_obj.varying_parameters().names())

print(ns_results_sep_adv.logz[-1], ns_results_sep_adv.logzerr[-1])

804.8397006212034 0.4608518659062479
